# Kalman Filter

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from numpy.lib.stride_tricks import as_strided as strided
import cv2
import numpy as np

In [3]:
coordinates = pd.read_csv('3d_coordinates.csv')
coordinates = coordinates.rename(columns={'Unnamed: 0':'frames'}).drop(['frames'], axis=1)
coordinates

,frame,x,y,z,bodypart
0,0.0,NaN,NaN,NaN,elbow
1,0.0,0.081465,-0.337530,-0.014157,wrist
2,0.0,0.002811,-0.309059,-0.007110,thumb1
3,0.0,-0.013208,-0.284525,-0.007068,thumb2
4,0.0,-0.021903,-0.260520,-0.009726,thumb3
...,...,...,...,...,...
10768,512.0,0.015968,-0.245930,-0.013913,ring4
10769,512.0,0.077863,-0.279638,-0.014540,little1
10770,512.0,0.055887,-0.256552,-0.005553,little2
10771,512.0,0.041653,-0.249463,-0.009889,little3


In [12]:
""" KALMAN FILTER """

measurement = np.zeros((3,1),dtype=np.float32)
state = np.zeros((9,1),dtype=np.float32)
kalman = cv2.KalmanFilter(9,3,0)

def initKalman(x,y,z):
    measurement[0][0] = x
    measurement[1][0] = y
    measurement[2][0] = y
    kalman.statePre = np.zeros((9,1),dtype=np.float32)
    kalman.statePre[0,0] = x
    kalman.statePre[1,0] = y
    kalman.statePre[2,0] = z
    kalman.statePost = np.zeros((9,1),dtype=np.float32)
    kalman.statePost[0,0] = x
    kalman.statePost[1,0] = y
    kalman.statePost[2,0] = z
    kalman.measurementMatrix=cv2.setIdentity(kalman.measurementMatrix)
    kalman.processNoiseCov=cv2.setIdentity(kalman.processNoiseCov, .01)
    kalman.measurementNoiseCov=cv2.setIdentity(kalman.measurementNoiseCov, .1)
    kalman.errorCovPost=cv2.setIdentity(kalman.errorCovPost, .1)
    dt=1/200
    v = dt
    a = 0.5*(dt**2)
    
    kalman.transitionMatrix = np.array([
                                [1, 0, 0, v, 0, 0, a, 0, 0],
                                [0, 1, 0, 0, v, 0, 0, a, 0],
                                [0, 0, 1, 0, 0, v, 0, 0, a],
                                [0, 0, 0, 1, 0, 0, v, 0, 0],
                                [0, 0, 0, 0, 1, 0, 0, v, 0],
                                [0, 0, 0, 0, 0, 1, 0, 0, v],
                                [0, 0, 0, 0, 0, 0, 1, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0, 1, 0],
                                [0, 0, 0, 0, 0, 0, 0, 0, 1]], np.float32)

def kalmanPredict():
    prediction = kalman.predict()
    predictPr = [prediction[0,0],prediction[1,0],prediction[2,0]]
    return predictPr

def kalmanCorrect(x,y,z):
    measurement[0,0] = x
    measurement[1,0] = y
    measurement[2,0] = z
    estimated = kalman.correct(measurement)
    return [estimated[0,0],estimated[1,0],estimated[2,0]]

def mask_knans(a, x):
    a = np.asarray(a)
    k = a.size
    n = np.append(np.isnan(a), [False] * (x - 1))
    m = np.empty(k, np.bool8)
    m.fill(True)

    s = n.strides[0]
    i = np.where(strided(n, (k + 1 - x, x), (s, s)).all(1))[0][:, None]
    i = i + np.arange(x)
    i = pd.unique(i[i < k])

    m[i] = False

    return m

def fill_nan(A):
    '''
    interpolate to fill nan values
    '''
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good],bounds_error=False)
    B = np.where(~mask_knans(A,3),A,f(inds))
    return B

initialized=False

bodyparts=pd.unique(coordinates['bodypart'])
filtered_coordinates=[]
# browse the bodyparts
for bodypart in bodyparts:
    corrected=[]
    #filter coordinates by bodypart
    single_bp = coordinates[coordinates['bodypart'] == bodypart].loc[:,['x','y','z']]
    single_bp = np.array(single_bp)
    for idx in range(len(single_bp)):
        coordinate=single_bp[idx]
        if not np.any(np.isnan(coordinate)):
            if not initialized:
                initKalman(coordinate[0],coordinate[1],coordinate[2])
                initialized=True
                corrected.append(coordinate)
            else:
                p = kalmanPredict()
                s = kalmanCorrect(coordinate[0],coordinate[1],coordinate[2]);   
                corrected.append(s)
        else:
            corrected.append(coordinate)
    corrected=np.vstack(corrected)

    for i in range(3):
        corrected[:,i]=fill_nan(corrected[:,i])
        
    filtered_coordinates.append(corrected)

In [61]:
kal = coordinates.sort_values(by=['bodypart', 'frame'])
kal.index = kal['frame']
kal

,frame,x,y,z,bodypart
frame,,,,,
0.0,0.0,NaN,NaN,NaN,elbow
1.0,1.0,NaN,NaN,NaN,elbow
2.0,2.0,NaN,NaN,NaN,elbow
3.0,3.0,NaN,NaN,NaN,elbow
4.0,4.0,NaN,NaN,NaN,elbow
...,...,...,...,...,...
508.0,508.0,0.081528,-0.344655,-0.032526,wrist
509.0,509.0,0.081479,-0.344670,-0.032602,wrist
510.0,510.0,0.081453,-0.344681,-0.032621,wrist


In [56]:
kalman_coordinates = pd.DataFrame.from_records(data=np.vstack(filtered_coordinates), columns=['x', 'y', 'z'])
# kalman_coordinates['frame'] = coordinates['frame'].reset_index(drop=True)
kalman_coordinates.index = list(kal['frame'].unique()) * 21
kalman_coordinates['frame'] = kalman_coordinates.index
kalman_coordinates['bodypart'] = kal['bodypart']
kalman_coordinates

,x,y,z,frame,bodypart
0.0,NaN,NaN,NaN,0.0,elbow
1.0,NaN,NaN,NaN,1.0,elbow
2.0,NaN,NaN,NaN,2.0,elbow
3.0,NaN,NaN,NaN,3.0,elbow
4.0,NaN,NaN,NaN,4.0,elbow
...,...,...,...,...,...
508.0,0.039586,-0.243323,-0.001672,508.0,wrist
509.0,0.039648,-0.243380,-0.001708,509.0,wrist
510.0,0.039731,-0.243406,-0.001640,510.0,wrist
511.0,0.039774,-0.243455,-0.001608,511.0,wrist
